# Databases and SQL
## Lecture objectives

1. Discuss when and why you would want to use a database, rather than `pandas`
2. Demonstrate how to set up a PostgreSQL and PostGIS database
3. Explore some basic SQL queries
4. Introduce how to integrate a database with Python and QGIS

Today's lecture will be mostly a taster. We won't do much on coding in SQL. Rather, I want to give you enough background so that you'll know when you should go further, and where to start.

If you are already familiar with GIS, [this primer by Helen McKenzie](https://www.helenmakesmaps.com/post/how-to-sql-a-guide-for-gis-users) is a very useful guide to transitioning to SQL.

One takeway: you should never need to pay money for ArcGIS. The open-source substitutes are fabulous! (But as long as most public agencies continue to think otherwise, ArcGIS is still an important skill for planners.)

## Why use a database
In most cases, Python in general, and `pandas`/`geopandas` in particular, will do an excellent job for your data science workflow. But sometimes, they might not be the best option. Here are some situations when you might want to consider a database.
* Your dataset won't fit in memory
* Multiple users need access at the same time
* You need to validate and maintain the integrity of the data through access controls and transactions (a transaction means that if the computer crashes halfway through, your data integrity is unaffected)

Many planning agencies have databases already hosted on the agency's server, for example to store land-use permits or GIS files. Some of these might be spatial databases, e.g. with ESRI's ArcGIS suite. You might need to query these as part of your work.

You can also set up your own database, even on a laptop. External hard drives are pretty cheap. Of course, if you want someone else to access it, an always-on computer with a stable internet connection is needed. But this could even be an old computer that you keep in the corner of the office or lab.

**Key takeaway:** Data that needs to be preserved and maintained, whether permit issuance, rider surveys, or asset inventories such as bus stops, shouldn't be an Excel file on someone's computer. What happens if they retire or their computer gets stolen? Or someone inadvertently "corrects" the data and introduces errors? Or different users have different versions of the Excel file on their computers?

## How to set up a database
I highly recommend PostgreSQL (Postgres for short). It's free, open-source, and cross-platform (runs on Windows, Mac, and Linux.) It is well supported and updated, and has incredible spatial analysis capabilities through the [PostGIS](http://postgis.net) extension.

Most databases use the same language — SQL, which stands for Structured Query Language. So if you learn Postgres, you should be able to use pretty much any alternative with minimal hassle.

Installing Postgres is simple. We won't need to do it for this class. But if you feel inclined to experiment, [the PostGIS site provides a list of downloadable versions](http://postgis.net/install/). For Mac, I recommend postgres.app to get started, as it works just like any Mac app, and the Homebrew version for more advanced users.

You don't need any sort of fancy computer to download and run Postgres. If you have large datasets or intensive computations, almost any cloud provider (e.g. AWS) lets you run Postgres. But for most purposes, a standard laptop or desktop computer will work well.

## Database structure and access
Postgres organizes your data into:
* Databases. For example, I usually have one database per research project.
* Schemas. These are like directories on your computer, to organize your tables. For example, you might have schemas for `inputdata`, `working`, and `output`.
* Tables. Each Postgres table is like a `(geo)pandas` `DataFrame`.

How do you access the data? Here are some options:
* The graphical interface `pgAdmin 4`, which runs in a web browser
* The command line (`psql`)
* If you have spatial data, [QGIS](https://qgis.org/en/site/), an open-source alternative to ArcMap
* And of course...Python. `pandas` can read and write database tables with the `read_sql_query` and `to_sql` functions. For more flexibility, the `psycopg2` (soon-to-be `psycopg3`) library provides more power.

## Basic SQL
Even if you don't set up your own database, it can be useful to know how to query a database using SQL. The intuition is pretty similar to `pandas` — particularly with joins and `groupby` functions.

Here, we will walk through some practical examples. But rather than querying a database using SQL, we'll query a `pandas` dataframe using SQL, using `pandasql`. 

We'll use the same Ventura County collisions data from a couple of weeks ago. [Here's a refresher on the data structure.](https://tims.berkeley.edu/help/SWITRS.php)

In [ ]:
import pandas as pd
from pandasql import PandaSQL
pdsql = PandaSQL()

collisionDf = pd.read_csv('../data/Collisions.csv')
partiesDf   = pd.read_csv('../data/Parties.csv')
victimsDf   = pd.read_csv('../data/Victims.csv')

# let's convert the columns to lower case to make them easier to read
# also, in Postgres, all column names are lower case
for df in [collisionDf, partiesDf, victimsDf]:
    df.columns = df.columns.str.lower()

In [ ]:
collisionDf.head()

### SELECT...FROM
The simplest SQL queries take this form:

`SELECT column1, column2...FROM tablename;`

Note the `;` at the end of each query.

For example, suppose we want to get two columns from `collisionDf`: `collision_date` and `officer_id`.

In [ ]:
pdsql('SELECT collision_date, officer_id FROM collisionDf;').head()

The equivalent in `pandas`: 

In [ ]:
collisionDf[['collision_date', 'officer_id']].head()

### WHERE
What if we only want to get a subset of rows? Use `WHERE`.

`SELECT column1, column2...FROM tablename WHERE condition;`

For example, suppose we want to get the two columns where a pedestrian is involved. From the documentation, this is any value except `A` in the `ped_action` column.

In [ ]:
pdsql('SELECT collision_date, officer_id FROM collisionDf WHERE ped_action!="A";').head()

The equivalent in `pandas`:

In [ ]:
collisionDf[collisionDf.ped_action!='A'][['collision_date', 'officer_id']].head()

### Other useful queries
* Rather than a column name, `*` gives you all columns
* `DISTINCT` gives unique values
* `LIMIT` limits the number of rows returned (e.g. `LIMIT 10`)
* `COUNT(*)` gives the number of rows

In [ ]:
pdsql('SELECT DISTINCT officer_id FROM collisionDf;').head()

In `pandas`, I'd probably use `unique()`.

In [ ]:
collisionDf.officer_id.unique()[:5]

In [ ]:
pdsql('SELECT DISTINCT officer_id FROM collisionDf LIMIT 2;')

<div class="alert alert-block alert-info">
    <strong>Exercise:</strong> Write a SQL query to report the number of collisions that officer 3386 was involved in.
</div>

In [ ]:
pdsql('SELECT COUNT(*) FROM collisionDf WHERE officer_id=3386;')

### GROUP BY

`GROUP BY` works the same way, at least conceptually as in `pandas`

In [ ]:
pdsql('SELECT juris, COUNT(*) AS n FROM collisionDf GROUP BY juris;')

In `pandas`:

In [ ]:
collisionDf.groupby('juris').size()

### JOINS
Finally, we can join tables together. Normally, to save disk space, we'd do this on the fly, rather than creating new joined tables.

The join concepts, such as left joins, inner joins, 1:1 joins, are the same as in `pandas`.

In [ ]:
# note the ''' allows a string to go over multiple lines, for readability
pdsql('''SELECT *
         FROM collisionDf 
         LEFT JOIN partiesDf USING (case_id);''').head()

### Spatial operations
PostGIS supports all the spatial operations that are in `geopandas`, and many more. For example:

`SELECT * FROM parcels, cities WHERE ST_Intersects(parcels.geom, cities.geom);`

does a spatial join on the two tables, using the `geom` (geometry) column.

Unfortunately they aren't implemented in `pandasql`, so we can't practice them here.

## A Python-based stack
We could have done all of these operations just as easily in basic `pandas`. Normally, there's no need to use `pandasql`. The point here is to demonstrate the SQL syntax, so that you can query other databases. 

A broader point is that Python is ideally suited to gluing together different operations. Technical and scientific software has Python "bindings" — you can access their functionality from within Python. For example:
* Postgres and other databases, via `psycopg2`
* Statistical software: Stata and R (use the `rpy2` library), or Stan (use `pystan`) if you are a Bayesian
* GIS software: QGIS, ArcGIS
* System-level functions like copying files and opening a web browser

## For more information
I highly recommend PostGIS in Action as an excellent introduction to PostGIS, with some basic background on PostgreSQL. [The UCLA library has a hard copy and ebook.](https://ucla.on.worldcat.org/oclc/910352485) For PostgreSQL and SQL more generally, there are many free online courses, [such as this one at UC Davis](https://www.coursera.org/specializations/learn-sql-basics-data-science).

<div class="alert alert-block alert-info">
<h3>Key Takeaways</h3>
<ul>
  <li>Databases are more complicated to work with than pandas dataframes. But they are a good solution if you have a lot of data, more than one concurrent user, or requirements for data validation and integrity.</li>
  <li>SQL uses different syntax, but the conceptual framework is similar to querying a pandas dataframe.</li>
  <li>Python can be used as a glue that holds your data collection and analysis pipeline together.</li>
</ul>
</div>